## **Data Wrangling**

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)  # This will auto-load the project path

In [3]:
from src.loaders.load_data import DataLoader
from src.loaders.config_loader import get_config

config = get_config()
loader = DataLoader()
df = loader.load_data(config['data']['raw_dir'])

Loading CSV file from: ../data/raw/saudi_store_sales_dataset.csv


In [4]:
from src.wranglers.dynamic_wrangler_selecter import get_wrangler  # create this function
wrangler = get_wrangler(df)


Using Spark DataFrame wrangler


In [5]:
wrangler = wrangler.clean_and_preprocess()

Starting Data Cleaning and Preprocessing...
Dropped columns: ['Invoice ID', 'Customer Type', 'Customer Name', 'Customer Gender', 'Employee Name', 'Product Name', 'Manager Name', 'Channel', 'Customer Satisfaction']
Current columns: ['Invoice Date', 'City', 'Product Category', 'Total Sales']
 Missing values handled
 Duplicates removed: 51 rows dropped
 Whitespace trimmed in 'City' and 'Category'
 Converted text to lowercase for 'City' and 'Product Category'
 Removed special characters from 'City' and 'Product Category'
 Final DataFrame schema:
root
 |-- Invoice Date: string (nullable = true)
 |-- City: string (nullable = false)
 |-- Product Category: string (nullable = false)
 |-- Total Sales: integer (nullable = true)

Total Rows: 49947 | Total Columns: 4
 Data Cleaning and Preprocessing Complete!


In [6]:
wrangler.df.show(5)

+------------+---------------+----------------+-----------+
|Invoice Date|           City|Product Category|Total Sales|
+------------+---------------+----------------+-----------+
|    1/4/2020|          mecca|    boys fashion|        195|
|    1/7/2020|         riyadh|  womens fashion|       3045|
|   1/18/2020|wadi ad dawasir|    pet supplies|        552|
|   1/24/2020|          tabuk|    boys fashion|        204|
|    2/6/2020|         riyadh|         luggage|       6806|
+------------+---------------+----------------+-----------+
only showing top 5 rows



In [7]:
wrangler = wrangler.validate_schema()

In [8]:
wrangler.df.show(5)

+------------+---------------+----------------+-----------+
|Invoice Date|           City|Product Category|Total Sales|
+------------+---------------+----------------+-----------+
|    1/4/2020|          mecca|    boys fashion|      195.0|
|    1/7/2020|         riyadh|  womens fashion|     3045.0|
|   1/18/2020|wadi ad dawasir|    pet supplies|      552.0|
|   1/24/2020|          tabuk|    boys fashion|      204.0|
|    2/6/2020|         riyadh|         luggage|     6806.0|
+------------+---------------+----------------+-----------+
only showing top 5 rows



In [9]:
wrangler = wrangler.detect_and_handle_outliers()

Detecting outliers for column 'Total Sales':
   Lower Bound: -3429.0, Upper Bound: 6723.0
   Outliers detected: 1516
 Outliers handled for column 'Total Sales'.


In [10]:
wrangler.df.show(5)

+------------+---------------+----------------+-----------+
|Invoice Date|           City|Product Category|Total Sales|
+------------+---------------+----------------+-----------+
|    1/4/2020|          mecca|    boys fashion|      195.0|
|    1/7/2020|         riyadh|  womens fashion|     3045.0|
|   1/18/2020|wadi ad dawasir|    pet supplies|      552.0|
|   1/24/2020|          tabuk|    boys fashion|      204.0|
|    2/6/2020|         riyadh|         luggage|     6723.0|
+------------+---------------+----------------+-----------+
only showing top 5 rows



In [11]:
wrangler = wrangler.aggregate_for_forecasting()

+---------------+----------+-----------------+
|City           |Week Start|City Weekly Sales|
+---------------+----------+-----------------+
|mecca          |2019-12-30|25070.0          |
|alula          |2019-12-30|15265.0          |
|buraydah       |2019-12-30|46717.0          |
|hafar al batin |2019-12-30|30144.0          |
|medina         |2019-12-30|46691.0          |
|wadi ad dawasir|2019-12-30|28302.0          |
|riyadh         |2019-12-30|51725.0          |
|arar           |2019-12-30|13390.0          |
|tabuk          |2019-12-30|64394.0          |
|najran         |2019-12-30|23524.0          |
|jeddah         |2019-12-30|32675.0          |
|al khobar      |2019-12-30|12737.0          |
|al khobar      |2020-01-06|14225.0          |
|najran         |2020-01-06|39851.0          |
+---------------+----------+-----------------+
only showing top 14 rows

+------------------------+----------+---------------------+
|Product Category        |Week Start|Category Weekly Sales|
+-------

In [12]:
wrangler.df.show(5)

+------------+---------------+----------------+-----------+-------------------+
|Invoice Date|           City|Product Category|Total Sales|         Week Start|
+------------+---------------+----------------+-----------+-------------------+
|  2020-01-04|          mecca|    boys fashion|      195.0|2019-12-30 00:00:00|
|  2020-01-07|         riyadh|  womens fashion|     3045.0|2020-01-06 00:00:00|
|  2020-01-18|wadi ad dawasir|    pet supplies|      552.0|2020-01-13 00:00:00|
|  2020-01-24|          tabuk|    boys fashion|      204.0|2020-01-20 00:00:00|
|  2020-02-06|         riyadh|         luggage|     6723.0|2020-02-03 00:00:00|
+------------+---------------+----------------+-----------+-------------------+
only showing top 5 rows



In [13]:
wrangler = wrangler.run_quality_checks()

Null Counts:
+------------+----+----------------+-----------+----------+
|Invoice Date|City|Product Category|Total Sales|Week Start|
+------------+----+----------------+-----------+----------+
|           0|   0|               0|          0|         0|
+------------+----+----------------+-----------+----------+

Data Types:
Invoice Date: Expected=date, Actual=DateType()
City: Expected=string, Actual=StringType()
Product Category: Expected=string, Actual=StringType()
Total Sales: Expected=float, Actual=DoubleType()

 Checking for Negative or Zero Sales:
+------------+----+----------------+-----------+----------+
|Invoice Date|City|Product Category|Total Sales|Week Start|
+------------+----+----------------+-----------+----------+
+------------+----+----------------+-----------+----------+


 Checking for Duplicates:
Found 6 duplicate rows.
+------------+---------+----------------+-----------+-------------------+-----+
|Invoice Date|     City|Product Category|Total Sales|         Week St

In [14]:
wrangler = wrangler.export_and_version_data()

Directory already exists: ../data/processed

Exporting the main processed DataFrame...
Main Data saved as: ../data/processed\main_data_v2.csv

Exporting the city-level weekly sales DataFrame...
City Weekly Sales saved as: ../data/processed\city_sales_weekly_v1.csv

Exporting the category-level weekly sales DataFrame...
Category Weekly Sales saved as: ../data/processed\category_sales_weekly_v1.csv

All exports completed successfully.

You can check the processed data here: ../data/processed
